<h1>Liveability</h1>

In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, '../scripts/')
from helper_functions import convert_census_to_postcode

import geopandas as gpd 
import folium

Obtaining property data

In [3]:
# Reading in preprocessed property data
property_df = pd.read_csv("../data/curated/properties_processed.csv")

# Remove unecessary columns "Unnamed: 0", "Name", "Coordinates", "Property_Type" & "Agency"
property_df = property_df.drop(columns = ["Unnamed: 0", "Name", "Coordinates", "Property_Type", "Agency"])
property_df.head()

,Cost,Bed,Bath,Parking,Postcode
0,440.0,1,1,0,3000
1,620.0,1,1,0,3000
2,300.0,1,1,0,3000
3,400.0,1,1,0,3000
4,625.0,2,2,1,3000


In [4]:
# Reading in preprocessed distance data
distances_df = pd.read_csv("../data/curated/catergorised_distances.csv")

# Removing unecessary columns (all columns apart from numerical/categorical distance measures & postcode)
distances_df = distances_df.drop(columns = ["Unnamed: 0", "index", "Name", "Cost", "Coordinates", "Bed", "Bath", "Parking", "Property_Type", "Agency"])
distances_df.head()

,Postcode,cbd_distance,station_distance,park_distance,postoffice_distance
0,3000,749.2,2,1,1
1,3000,951.3,2,1,1
2,3000,577.3,1,2,2
3,3000,846.9,3,5,1
4,3000,1052.5,1,2,1


Calculating the average distances

In [5]:
# Calculating the average distance from the CBD per postcode
avg_cbd_dist = distances_df.groupby(by = "Postcode")["cbd_distance"].mean()
avg_cbd_dist.head()

Postcode
3000    1083.888732
3002    2436.186364
3003    1811.239683
3004    4058.924731
3006    2456.527692
Name: cbd_distance, dtype: float64

In [6]:
# Calculating the average number of amenities per postcode
num_station = distances_df.groupby(by = "Postcode")["station_distance"].mean()
num_station.head()

Postcode
3000    1.873239
3002    2.250000
3003    1.968254
3004    3.967742
3006    3.779487
Name: station_distance, dtype: float64

In [7]:
num_park = distances_df.groupby(by = "Postcode")["park_distance"].mean()
num_park.head()

Postcode
3000    1.781690
3002    1.204545
3003    1.253968
3004    1.860215
3006    1.482051
Name: park_distance, dtype: float64

In [8]:
num_post = distances_df.groupby(by = "Postcode")["postoffice_distance"].mean()
num_post.head()

Postcode
3000    1.528169
3002    2.363636
3003    2.158730
3004    1.913978
3006    2.815385
Name: postoffice_distance, dtype: float64

In [9]:
# Creating dataframe of average amenities per postcode
amenities_df = pd.DataFrame()
amenities_df["Postcode"] = property_df["Postcode"].unique()
amenities_df["Average Distance to CBD"] = avg_cbd_dist.to_list()
amenities_df["Average # Train Station"] = num_station.to_list()
amenities_df["Average # Park"] = num_park.to_list()
amenities_df["Average # Post Office"] = num_post.to_list()
amenities_df.head()

,Postcode,Average Distance to CBD,Average # Train Station,Average # Park,Average # Post Office
0,3000,1083.888732,1.873239,1.781690,1.528169
1,3002,2436.186364,2.250000,1.204545,2.363636
2,3003,1811.239683,1.968254,1.253968,2.158730
3,3004,4058.924731,3.967742,1.860215,1.913978
4,3006,2456.527692,3.779487,1.482051,2.815385


Calculating the average number of facilities

In [10]:
# Calculating the average number of facilities per postcode
avg_bed = property_df.groupby(by = "Postcode")["Bed"].mean()
avg_bed.head()

Postcode
3000    1.601399
3002    1.909091
3003    1.796875
3004    1.763441
3006    1.785714
Name: Bed, dtype: float64

In [11]:
avg_bath = property_df.groupby(by = "Postcode")["Bath"].mean()
avg_bath.head()

Postcode
3000    1.300699
3002    1.318182
3003    1.343750
3004    1.494624
3006    1.469388
Name: Bath, dtype: float64

In [12]:
avg_parking = property_df.groupby(by = "Postcode")["Parking"].mean()
avg_parking.head()

Postcode
3000    0.356643
3002    1.000000
3003    0.609375
3004    1.096774
3006    0.647959
Name: Parking, dtype: float64

In [13]:
# Creating dataframe of average facilities per postcode
facilities_df = pd.DataFrame()
facilities_df["Postcode"] = property_df["Postcode"].unique()
facilities_df["Average # Beds"] = avg_bed.to_list()
facilities_df["Average # Baths"] = avg_bath.to_list()
facilities_df["Average # Parking"] = avg_parking.to_list()
facilities_df.head()

,Postcode,Average # Beds,Average # Baths,Average # Parking
0,3000,1.601399,1.300699,0.356643
1,3002,1.909091,1.318182,1.000000
2,3003,1.796875,1.343750,0.609375
3,3004,1.763441,1.494624,1.096774
4,3006,1.785714,1.469388,0.647959


In [14]:
# Joining facilities and amenities dataframes
amenities_df.set_index("Postcode", inplace = True)
facilities_df.set_index("Postcode", inplace = True)
postcode_property_df = amenities_df.join(facilities_df, on = "Postcode")
postcode_property_df.head()

,Average Distance to CBD,Average # Train Station,Average # Park,Average # Post Office,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,,
3000,1083.888732,1.873239,1.781690,1.528169,1.601399,1.300699,0.356643
3002,2436.186364,2.250000,1.204545,2.363636,1.909091,1.318182,1.000000
3003,1811.239683,1.968254,1.253968,2.158730,1.796875,1.343750,0.609375
3004,4058.924731,3.967742,1.860215,1.913978,1.763441,1.494624,1.096774
3006,2456.527692,3.779487,1.482051,2.815385,1.785714,1.469388,0.647959


Obtaining census/population data

In [15]:
# Reading in csv files for obtaining census data
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")
sa2_postcode_map.set_index("sa2_2021", inplace = True)
census_df = pd.read_csv("../data/curated/census_data.csv")

In [16]:
# Using helper function to convert sa2 mapping to postcodes
census_by_postcode_df = convert_census_to_postcode(census_df, sa2_postcode_map, "mean_no_zero")
census_by_postcode_df.head()

,postcode_2021,tot_population_11,tot_population_16,tot_population_21,avg_med_mortg_rep_11,avg_med_mortg_rep_16,avg_med_mortg_rep_21,avg_med_person_inc_11,avg_med_person_inc_16,avg_med_person_inc_21,avg_med_rent_16,avg_med_rent_11,avg_med_rent_21,avg_med_hh_inc_16,avg_med_hh_inc_11,avg_med_hh_inc_21,tot_avg_hh_size_16,tot_avg_hh_size_11,tot_avg_hh_size_21
0,3000,124551,167166,178424,2213.38,2040.38,2040.19,862.18,5483.82,6467.76,395.76,447.06,418.19,1482.53,1896.76,2159.41,1.88,1.97,1.86
1,3002,68729,82804,89023,2357.78,2173.67,2155.22,1091.80,8969.60,10432.90,398.00,460.33,449.67,1709.40,2415.00,2598.80,1.82,1.91,1.87
2,3003,15496,20633,23083,2200.00,2050.00,2085.00,701.50,716.00,1000.00,395.00,418.50,385.50,1466.00,1493.50,1751.00,2.15,2.15,1.95
3,3004,100879,123254,129273,2331.58,2155.67,2149.75,1066.08,7152.46,8339.46,391.15,446.83,440.75,1688.85,2270.46,2471.46,1.83,1.89,1.84
4,3006,21150,30239,36164,2477.25,2217.75,2079.00,1132.40,16783.00,19507.00,406.80,501.00,461.00,1637.20,2883.20,3088.80,1.80,1.92,1.92


In [17]:
# Removing unnecessary columns from census data (only need columns containing data from 2021)
census_by_postcode_df = census_by_postcode_df[["postcode_2021", "tot_population_21", "avg_med_mortg_rep_21", "avg_med_person_inc_21", "avg_med_rent_21", 
                            "avg_med_hh_inc_21", "tot_avg_hh_size_21"]]

# Selecting columns required for assessing liveability
population_df = census_by_postcode_df[["postcode_2021", "tot_avg_hh_size_21"]]
population_df.head()

,postcode_2021,tot_avg_hh_size_21
0,3000,1.86
1,3002,1.87
2,3003,1.95
3,3004,1.84
4,3006,1.92


In [18]:
# Renaming postcode column of census dataframe and setting to index
population_df = population_df.rename({"postcode_2021": "Postcode"}, axis = 1)
population_df = population_df.set_index("Postcode")
population_df.head()

,tot_avg_hh_size_21
Postcode,
3000,1.86
3002,1.87
3003,1.95
3004,1.84
3006,1.92


In [19]:
# Joining census population dataframe and facilities and amenities dataframe
df = population_df.join(postcode_property_df, on = "Postcode")
df.head()

,tot_avg_hh_size_21,Average Distance to CBD,Average # Train Station,Average # Park,Average # Post Office,Average # Beds,Average # Baths,Average # Parking
Postcode,,,,,,,,
3000,1.86,1083.888732,1.873239,1.781690,1.528169,1.601399,1.300699,0.356643
3002,1.87,2436.186364,2.250000,1.204545,2.363636,1.909091,1.318182,1.000000
3003,1.95,1811.239683,1.968254,1.253968,2.158730,1.796875,1.343750,0.609375
3004,1.84,4058.924731,3.967742,1.860215,1.913978,1.763441,1.494624,1.096774
3006,1.92,2456.527692,3.779487,1.482051,2.815385,1.785714,1.469388,0.647959


Calculating liveability metric

In [20]:
# If the number of facilities is 0 (i.e. for number of bedrooms or number of parking spaces) replace 0 value with arbitrary small value epsilon
EPSILON = 10**(-6)
 
df.loc[df["Average # Beds"] == 0, "Average # Beds"] = EPSILON
df.loc[df["Average # Parking"] == 0, "Average # Parking"] = EPSILON

In [21]:
TOTAL_PROPERTIES = len(property_df)

# Counting the number of properties per postcode
num_properties = property_df.groupby("Postcode").size()

In [22]:
# Calcuting the contribution of each postcode to the total number of rental properties as a proportion
df["Property Proportion"] = num_properties / TOTAL_PROPERTIES
df.head()

,tot_avg_hh_size_21,Average Distance to CBD,Average # Train Station,Average # Park,Average # Post Office,Average # Beds,Average # Baths,Average # Parking,Property Proportion
Postcode,,,,,,,,,
3000,1.86,1083.888732,1.873239,1.781690,1.528169,1.601399,1.300699,0.356643,0.010816
3002,1.87,2436.186364,2.250000,1.204545,2.363636,1.909091,1.318182,1.000000,0.003328
3003,1.95,1811.239683,1.968254,1.253968,2.158730,1.796875,1.343750,0.609375,0.004841
3004,1.84,4058.924731,3.967742,1.860215,1.913978,1.763441,1.494624,1.096774,0.007034
3006,1.92,2456.527692,3.779487,1.482051,2.815385,1.785714,1.469388,0.647959,0.014825


In [23]:
# Calculating the average number of facilities per person in a household (for each postcode)
df["Beds per Person"] = df["Average # Beds"] / df["tot_avg_hh_size_21"]
df["Baths per Person"] = df["Average # Baths"] / df["tot_avg_hh_size_21"]
df["Parking per Person"] = df["Average # Parking"] / df["tot_avg_hh_size_21"]
df.head()

,tot_avg_hh_size_21,Average Distance to CBD,Average # Train Station,Average # Park,Average # Post Office,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Beds per Person,Baths per Person,Parking per Person
Postcode,,,,,,,,,,,,
3000,1.86,1083.888732,1.873239,1.781690,1.528169,1.601399,1.300699,0.356643,0.010816,0.860967,0.699301,0.191744
3002,1.87,2436.186364,2.250000,1.204545,2.363636,1.909091,1.318182,1.000000,0.003328,1.020904,0.704910,0.534759
3003,1.95,1811.239683,1.968254,1.253968,2.158730,1.796875,1.343750,0.609375,0.004841,0.921474,0.689103,0.312500
3004,1.84,4058.924731,3.967742,1.860215,1.913978,1.763441,1.494624,1.096774,0.007034,0.958392,0.812295,0.596073
3006,1.92,2456.527692,3.779487,1.482051,2.815385,1.785714,1.469388,0.647959,0.014825,0.930060,0.765306,0.337479


In [24]:
# Calculating non-stadardised liveability metric
df["Liveability"] = ((1 / df["Average Distance to CBD"]) + df["Average # Train Station"] + df["Average # Park"] + df["Average # Post Office"] 
                        + (df["Property Proportion"] * 100) + df["Beds per Person"] + df["Baths per Person"] + df["Parking per Person"])
df.head()

,tot_avg_hh_size_21,Average Distance to CBD,Average # Train Station,Average # Park,Average # Post Office,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Beds per Person,Baths per Person,Parking per Person,Liveability
Postcode,,,,,,,,,,,,,
3000,1.86,1083.888732,1.873239,1.781690,1.528169,1.601399,1.300699,0.356643,0.010816,0.860967,0.699301,0.191744,8.017645
3002,1.87,2436.186364,2.250000,1.204545,2.363636,1.909091,1.318182,1.000000,0.003328,1.020904,0.704910,0.534759,8.411970
3003,1.95,1811.239683,1.968254,1.253968,2.158730,1.796875,1.343750,0.609375,0.004841,0.921474,0.689103,0.312500,7.788660
3004,1.84,4058.924731,3.967742,1.860215,1.913978,1.763441,1.494624,1.096774,0.007034,0.958392,0.812295,0.596073,10.812368
3006,1.92,2456.527692,3.779487,1.482051,2.815385,1.785714,1.469388,0.647959,0.014825,0.930060,0.765306,0.337479,11.592665


In [25]:
# Calculating standardised liveability metric
min_liveability = df.sort_values(by = "Liveability").head(1)["Liveability"].tolist()[0]
#print(min_liveability)
max_liveability = df.sort_values(by = "Liveability", ascending = False).head(1)["Liveability"].tolist()[0]
#print(max_liveability)

df["Standardised Liveability"] = (df["Liveability"] - min_liveability) / (max_liveability - min_liveability)

# Printing the top 10 most liveable suburbs
df.sort_values(by = "Standardised Liveability", ascending = False).head(10)

,tot_avg_hh_size_21,Average Distance to CBD,Average # Train Station,Average # Park,Average # Post Office,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Beds per Person,Baths per Person,Parking per Person,Liveability,Standardised Liveability
Postcode,,,,,,,,,,,,,,
3213,2.49,68820.85,5.0,4.0,5.0,3.5,1.75,9.25,0.000303,1.405622,0.702811,3.714859,19.853563,1.000000
3688,2.40,297356.40,5.0,5.0,5.0,5.0,3.00,3.00,0.000076,2.083333,1.250000,1.250000,19.590900,0.982630
3331,2.49,92937.65,5.0,4.5,5.0,3.5,2.00,6.50,0.000151,1.405622,0.803213,2.610442,19.334415,0.965669
3808,2.82,56243.80,5.0,5.0,5.0,5.0,4.00,2.00,0.000076,1.773050,1.418440,0.709220,18.908291,0.937490
3670,2.40,189962.30,5.0,5.0,5.0,4.0,1.00,4.00,0.000076,1.666667,0.416667,1.666667,18.757569,0.927523
3211,2.70,50868.20,5.0,5.0,5.0,4.0,2.00,4.00,0.000076,1.481481,0.740741,1.481481,18.711287,0.924462
3757,2.80,52128.30,5.0,5.0,5.0,4.0,2.00,4.00,0.000076,1.428571,0.714286,1.428571,18.579011,0.915715
3878,2.25,289742.20,5.0,5.0,5.0,4.0,2.00,2.00,0.000076,1.777778,0.888889,0.888889,18.563123,0.914664
3799,2.30,82884.90,5.0,5.0,5.0,3.0,3.00,2.00,0.000076,1.304348,1.304348,0.869565,18.485837,0.909553


In [26]:
# Printing the top 10 least liveable postcodes
df.sort_values(by = "Standardised Liveability").head(10)

,tot_avg_hh_size_21,Average Distance to CBD,Average # Train Station,Average # Park,Average # Post Office,Average # Beds,Average # Baths,Average # Parking,Property Proportion,Beds per Person,Baths per Person,Parking per Person,Liveability,Standardised Liveability
Postcode,,,,,,,,,,,,,,
3783,2.90,69246.600000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000076,0.344828,0.344828,1.034483,4.731716,0.000000
3847,2.35,186706.000000,1.000000,1.000000,2.000000,3.000000,1.000000,2.000000,0.000076,1.276596,0.425532,0.851064,6.560761,0.120954
3099,2.85,35160.150000,2.000000,1.500000,1.500000,3.000000,1.500000,1.500000,0.000151,1.052632,0.526316,0.526316,7.120419,0.157964
3146,2.50,11932.350000,2.000000,1.666667,2.000000,1.833333,1.333333,1.166667,0.000454,0.733333,0.533333,0.466667,7.445466,0.179459
3003,1.95,1811.239683,1.968254,1.253968,2.158730,1.796875,1.343750,0.609375,0.004841,0.921474,0.689103,0.312500,7.788660,0.202154
3185,2.10,10804.552941,2.019608,1.431373,1.823529,2.019608,1.294118,1.215686,0.003857,0.961718,0.616246,0.578898,7.817215,0.204042
3373,2.49,160882.500000,3.000000,1.000000,2.000000,3.000000,1.000000,1.000000,0.000076,1.204819,0.401606,0.401606,8.015602,0.217162
3000,1.86,1083.888732,1.873239,1.781690,1.528169,1.601399,1.300699,0.356643,0.010816,0.860967,0.699301,0.191744,8.017645,0.217297
3056,2.15,5895.429412,2.218487,1.319328,1.747899,2.176471,1.226891,0.815126,0.009001,1.012312,0.570647,0.379128,8.148054,0.225921


Graphing standardised liveability

In [27]:
# Resetting dataframe index & selecting required subset of columns
df = df.reset_index()
liveability_df = df[["Postcode", "Standardised Liveability"]]

In [28]:
# Creating geoJSON file of postcode and geometry coordinates 
sf = gpd.read_file("../data/raw/POA_2021_AUST_GDA2020_SHP/POA_2021_AUST_GDA2020.shp")
postcodes = pd.read_csv("../data/raw/external/postcode.csv", names = ["POA_CODE21", "Name", "Area"])
postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)

# Converting the geometry shaape to to latitude and longitude
# TAKEN FROM TUTE 2 NOTEBOOK
sf["geometry"] = sf["geometry"].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf[sf["POA_CODE21"].between("3000", "4000")]
sf["POA_CODE21"] = sf["POA_CODE21"].astype(int)



gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

In [29]:
# (y, x) since we want (lat, long)
gdf["centroid"] = gdf["geometry"].apply(lambda x: (x.centroid.y, x.centroid.x))
gdf[["Name", "POA_CODE21", "centroid"]].head()

liveability_df.rename({"Postcode": "POA_CODE21"})

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

c = folium.Choropleth(
            geo_data = geoJSON, # geoJSON 
            name = "choropleth", # name of plot
            data = liveability_df, # data source
            columns = ["Postcode", "Standardised Liveability"], # the columns required
            key_on = "properties.POA_CODE21", # this is from the geoJSON's properties
            fill_color = "YlOrRd", # color scheme
            nan_fill_color = "grey",
            legend_name = "Liveability"
        )

c.add_to(m)
m.save(f"../plots/liveability_heatmap")

In [33]:
# Creating GeoPandas visualisation of top 10 most liveable postcodes
most_liveable_df = liveability_df.sort_values(by = "Standardised Liveability", ascending = False).head(10)

postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)
postcodes = postcodes.loc[postcodes["POA_CODE21"].isin(most_liveable_df["Postcode"].to_list())]

In [34]:
gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

m.add_child(folium.Choropleth(geo_data = geoJSON, name = "choropleth",))
m.save(f"../plots/most_liveable_heatmap")

In [39]:
# Creating GeoPandas visualisation of top 10 least liveable postcodes
least_liveable_df = liveability_df.sort_values(by = "Standardised Liveability").head(10)

postcodes = pd.read_csv("../data/raw/external/postcode.csv", names = ["POA_CODE21", "Name", "Area"])
postcodes["POA_CODE21"] = postcodes["POA_CODE21"].astype(int)
postcodes = postcodes.loc[postcodes["POA_CODE21"].isin(least_liveable_df["Postcode"].to_list())]

In [40]:
gdf = gpd.GeoDataFrame(
    pd.merge(postcodes, sf, on = "POA_CODE21", how = "inner")
)

geoJSON = gdf[["POA_CODE21", "geometry"]].drop_duplicates("POA_CODE21").to_json()

m = folium.Map(location = [-37.8136, 144.9631], tiles = "Stamen Terrain", zoom_start = 10)

m.add_child(folium.Choropleth(geo_data = geoJSON, name = "choropleth",))
m.save(f"../plots/least_liveable_heatmap")